In [1]:
# import necessary libraries
import mesa
import seaborn as sns
import pandas as pd
import shutil

# import cacheable model wrapper
from cacheable_model import CacheableModel

The following code is to clear the default directory which the cached fiels are saved to.

In [2]:
directory = "output_dir"

def delete_directory(directory_path):
    shutil.rmtree(directory_path)
    print(f"Directory {directory_path} and its contents have been deleted.")

try:
    delete_directory(directory)
except:
    pass

Directory output_dir and its contents have been deleted.


Define the Boltzmann Wealth Model. I chose to reimplement it here so that I can modify the attributes and experiment with the results more easily. The code is taken from the tutorial

In [3]:
def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        cellmates.pop(
            cellmates.index(self)
        )  # Ensure agent is not giving money to itself
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            if other == self:
                print("I JUST GAVE MONEY TO MYSELF HEHEHE!")

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

Instantiate the base model without caching

In [4]:
model = MoneyModel(100, 10, 10)

In [5]:
def condition_function(model_vars): # condition to cache the results specifically
        return model_vars.get('Gini', 0)[-1] > 0.7
    
number_of_steps = 1000

cacheable_model = CacheableModel(model, directory, number_of_steps, 100, condition_function=condition_function)

In [6]:
for i in range(number_of_steps):
    cacheable_model.model.step()
    cacheable_model.get_grid_dataframe("test")
    cacheable_model.cache()

Condition met. Appended special results for step 42 to output_dir/special_results.parquet
Condition met. Appended special results for step 43 to output_dir/special_results.parquet
Condition met. Appended special results for step 44 to output_dir/special_results.parquet
Condition met. Appended special results for step 48 to output_dir/special_results.parquet
Condition met. Appended special results for step 140 to output_dir/special_results.parquet
Condition met. Appended special results for step 141 to output_dir/special_results.parquet
Condition met. Appended special results for step 142 to output_dir/special_results.parquet
Condition met. Appended special results for step 144 to output_dir/special_results.parquet
Condition met. Appended special results for step 145 to output_dir/special_results.parquet
Condition met. Appended special results for step 146 to output_dir/special_results.parquet
Condition met. Appended special results for step 147 to output_dir/special_results.parquet
Con

In [7]:
model_df, agent_df = cacheable_model.combine_dataframes()
model_df, agent_df

(       Gini
 0    0.0000
 1    0.3512
 2    0.4262
 3    0.5440
 4    0.5576
 ..      ...
 995  0.7032
 996  0.6944
 997  0.7048
 998  0.6834
 999  0.6902
 
 [1000 rows x 1 columns],
               Wealth
 Step AgentID        
 0    0             1
      1             1
      2             1
      3             1
      4             1
 ...              ...
 999  95            1
      96            0
      97            5
      98            0
      99            0
 
 [100000 rows x 1 columns])

In [8]:
def agent_portrayal(agent):
    size = 10
    color = "tab:red"
    if agent.wealth > 0:
        size = 50
        color = "tab:blue"
    return {"size": size, "color": color}


def _draw_grid(space, space_ax, agent_portrayal):
    def portray(g):
        x = []
        y = []
        s = []  # size
        c = []  # color
        for i in range(g.width):
            for j in range(g.height):
                content = g._grid[i][j]
                if not content:
                    continue
                if not hasattr(content, "__iter__"):
                    # Is a single grid
                    content = [content]
                for agent in content:
                    data = agent_portrayal(agent)
                    x.append(i)
                    y.append(j)
                    if "size" in data:
                        s.append(data["size"])
                    if "color" in data:
                        c.append(data["color"])
        out = {"x": x, "y": y}
        # This is the default value for the marker size, which auto-scales
        # according to the grid area.
        out["s"] = (180 / max(g.width, g.height)) ** 2
        if len(s) > 0:
            out["s"] = s
        if len(c) > 0:
            out["c"] = c
        return out

    space_ax.set_xlim(-1, space.width)
    space_ax.set_ylim(-1, space.height)
    space_ax.scatter(**portray(space))

In [9]:
import pandas as pd
from typing import List, Any
from mesa.space import MultiGrid as Grid
from matplotlib.figure import Figure
import solara
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator

def reconstruct_grid(filename='grid_cache.parquet', *attributes_list):
    # Load the DataFrame from Parquet
    df = pd.read_parquet(filename)
    
    # Create a new Grid instance
    width = df['pos_x'].max() + 1  # Assuming positions start from 0
    height = df['pos_y'].max() + 1  # Assuming positions start from 0
    grid = Grid(width, height, False)
    
    # Add agents to the grid
    for _, row in df.iterrows():
        agent = MoneyAgent(row['unique_id'], MoneyModel(100, 10, 10))
        agent.wealth = row["wealth"]
        grid.place_agent(agent, (row['pos_x'], row['pos_y']))
    
    return grid


In [10]:
# Example of loading a grid from a Parquet file
  # Your model instance, or a placeholder if required
grid = reconstruct_grid('test')
space_fig = Figure()
space_ax = space_fig.subplots()
space = getattr(model, "grid", None)

In [11]:
df = pd.read_parquet("test")
df

,pos_x,pos_y,unique_id,wealth
0,0,0,77,1
1,0,1,58,1
2,0,3,69,3
3,0,4,97,5
4,0,6,12,0
...,...,...,...,...
95,9,4,6,0
96,9,5,44,0
97,9,6,43,0
98,9,7,94,0


In [12]:
_draw_grid(space, space_ax, agent_portrayal)
solara.FigureMatplotlib(space_fig, format="png")

Cannot show ipywidgets in text